In [2]:
import matplotlib as plt
import numpy as np
import pandas as pd

df=pd.read_excel("TESTING BD.xlsx",infer_datetime_format=True)
names=df.columns
df.columns=[0]
df_to_explore=df[0].str.split(",",expand=True)
df_to_explore.columns=names[0].split(",")

%%bash
libreoffice --headless --convert-to csv TESTING BD.xlsx  --outdir .
ls

# (1) Loading the data and exploring it


In [10]:
df=pd.read_excel("TESTING BD.xlsx")
names=df.columns
df.columns=[0]
df_to_explore=df[0].str.split(",",expand=True)
df_to_explore.columns=names[0].split(",")
int_type=["Customer Lifetime Value","Income", "Monthly Premium Auto",
 "Months Since Last Claim","Months Since Policy Inception",
 "Number of Open Complaints","Number of Policies","Total Claim Amount"]
for i in int_type:
    df_to_explore[i]=df_to_explore[i].astype(float)
df_to_explore["Effective To Date"]=pd.to_datetime(df_to_explore["Effective To Date"])

In [12]:
df_to_explore.nunique()

Customer                         9134
State                               5
Customer Lifetime Value          8041
Response                            2
Coverage                            3
Education                           5
Effective To Date                  59
EmploymentStatus                    5
Gender                              2
Income                           5694
Location Code                       3
Marital Status                      3
Monthly Premium Auto              202
Months Since Last Claim            36
Months Since Policy Inception     100
Number of Open Complaints           6
Number of Policies                  9
Policy Type                         3
Policy                              9
Renew Offer Type                    4
Sales Channel                       4
Total Claim Amount               5106
Vehicle Class                       6
Vehicle Size                        3
dtype: int64

# 1)

In [50]:
df_to_explore.groupby(["State","Sales Channel"]).last()

Customer Customer Lifetime Value Response  Coverage  \
State      Sales Channel                                                       
Arizona    Agent          BE25221             6182.618843       No  Extended   
           Branch         MZ84108             9853.773193       No  Extended   
           Call Center    QJ62033             2415.063816       No     Basic   
           Web            CY12922             2506.360995       No     Basic   
California Agent          RX91025               19872.262       No   Premium   
           Branch         UP19263             7524.442436       No  Extended   
           Call Center    Y167826             2611.836866       No  Extended   
           Web            LA72316             23405.98798       No     Basic   
Nevada     Agent          GB92679             8421.516848       No   Premium   
           Branch         LR92373             2839.945741       No     Basic   
           Call Center    XC83045             16460.07754       No  Extended   
           Web            PG67485             9048.983447      Yes     Basic   
Oregon     Agent          QM60216             8906.991433       No  Extended   
           Branch         VQ78876             4660.399573       No     Basic   
           Call Center    JD86991             5097.318657       No     Basic   
           Web            QL80621             2249.874803       No     Basic   
Washington Agent          HP42153             2351.878675       No     Basic   
           Branch         EI17022             12168.74492      Yes     Basic   
           Call Center    PD69398             21710.27538       No  Extended   
           Web            BU59722             5037.953998       No     Basic   

                                     Education Effective To Date  \
State      Sales Channel                                           
Arizona    Agent          High School or Below            2/7/11   
           Branch                     Bachelor            1/2/11   
           Call Center    High School or Below            2/6/11   
           Web                         College           1/31/11   
California Agent          High School or Below           1/31/11   
           Branch                      College            2/3/11   
           Call Center                 College           2/14/11   
           Web                        Bachelor           2/10/11   
Nevada     Agent          High School or Below            1/5/11   
           Branch                     Bachelor            1/7/11   
           Call Center                Bachelor            2/9/11   
           Web            High School or Below           1/24/11   
Oregon     Agent                       College           1/10/11   
           Branch                     Bachelor            2/3/11   
           Call Center                Bachelor           2/22/11   
           Web                        Bachelor           2/22/11   
Washington Agent          High School or Below           1/29/11   
           Branch                     Bachelor            2/5/11   
           Call Center                 College           2/26/11   
           Web            High School or Below            1/9/11   

                         EmploymentStatus Gender Income Location Code  ...  \
State      Sales Channel                                               ...   
Arizona    Agent            Medical Leave      M  20072      Suburban  ...   
           Branch                Employed      M  44006         Urban  ...   
           Call Center         Unemployed      M      0      Suburban  ...   
           Web                 Unemployed      M      0      Suburban  ...   
California Agent               Unemployed      M      0      Suburban  ...   
           Branch                Employed      M  21941      Suburban  ...   
           Call Center         Unemployed      M      0      Suburban  ...   
           Web                   Employed      M  71941         U

0        5
1       42
2       38
3       65
4       44
        ..
9129    89
9130    28
9131    37
9132     3
9133    90
Name: Months Since Policy Inception, Length: 9134, dtype: object

In [28]:
df_to_explore.dtypes

Customer                         object
State                            object
Customer Lifetime Value          object
Response                         object
Coverage                         object
Education                        object
Effective To Date                object
EmploymentStatus                 object
Gender                           object
Income                           object
Location Code                    object
Marital Status                   object
Monthly Premium Auto             object
Months Since Last Claim          object
Months Since Policy Inception    object
Number of Open Complaints        object
Number of Policies               object
Policy Type                      object
Policy                           object
Renew Offer Type                 object
Sales Channel                    object
Total Claim Amount               object
Vehicle Class                    object
Vehicle Size                     object
dtype: object

In [18]:
names[0].split(",")

['Customer',
 'State',
 'Customer Lifetime Value',
 'Response',
 'Coverage',
 'Education',
 'Effective To Date',
 'EmploymentStatus',
 'Gender',
 'Income',
 'Location Code',
 'Marital Status',
 'Monthly Premium Auto',
 'Months Since Last Claim',
 'Months Since Policy Inception',
 'Number of Open Complaints',
 'Number of Policies',
 'Policy Type',
 'Policy',
 'Renew Offer Type',
 'Sales Channel',
 'Total Claim Amount',
 'Vehicle Class',
 'Vehicle Size']